In [1]:
# use conda enviroment "SpliceAI" to run this script

In [2]:
import sys

# ask Python to look for other scripts but not in the default directory
sys.path.append('../2_SpliceAI prediction of original sequence/') # The sys.path is a list that contains directories where Python looks for modules and packages
from SpliceAI import *

#print(sys.path)

2024-08-01 11:15:09.449216: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import csv

def extract_sequences_from_CSV_file(CSV):
    
    dict = {}
    
    with open(CSV, 'r', newline='') as csvfile:
        
        csv_reader = csv.reader(csvfile)
        
        # Skip the header row
        next(csv_reader)
        
        # Iterate over the remaining rows
        for row in csv_reader:
            
            potential_combination = row[0]
            sequecne = row[1]
            
            #print(potential_combination)
            #print(sequecne)
            
            dict[potential_combination] = sequecne
    
    return dict

In [23]:
def calculate_splicing_probabilities_of_insert_sequences(dict_insert_sequences):
    
    mGL = "atggtgagcaagggcgaggagctgttcaccggggtggtgcccatcctggtcgagctggacggcgacgtaaacggccacaagttcagcgtccgcggcgagggcgagggcgatgccaccaacggcaagctgaccctgaagttcatctgcaccaccggcaagctgcccgtgccctggcccaccctcgtgaccaccttaggctacggcgtggcctgcttcgcccgctaccccgaccacatgaagcagcacgacttcttcaagtccgccatgcccgaaggctacgtccaggagcgcaccatctctttcaaggacgacggtacctacaagacccgcgccgaggtgaagttcgagggcgacaccctggtgaaccgcatcgtgctgaagggcatcgacttcaaggaggacggcaacatcctggggcacaagctggagtacaacttcaacagccacaaggtctatatcacggccgacaagcagaagaacggcatcaaggctaacttcaagacccgccacaacgttgaggacggcggcgtgcagctcgccgaccactaccagcagaacacccccatcggcgacggccccgtgctgctgcccgacaaccactacctgagccatcagtccaaactgagcaaagaccccaacgagaagcgcgatcacatggtcctgaaggagagggtgaccgccgccgggattacacatgacatggatgaactatataaataa"
    fifteen_mer_barcode = "NNNNNNNNNNNNNNN"
    before_insert = "CTACTACCACCACCACCACCACCCCTTCTGGGGTCACTCCATCCATGCTCTCTCCAGCCAGCCATCTCAAGGAGAAACATAGTTCAACTGAAAGACTCATGCTCTGATTGTGGTGGGGTGGGGATCCTTGGGAAGAATTACTCCCAAGAGTAACTCTCATTATCTCCTCCACAGAAAACACACAGCTTCCACAACTTCTCTGTTTTCTGTCAGTCCCCCAGTGGCCGCCCTTACACGTCTCCTACTTCAATGGTAGGGGCGGTTTATTTATTTATTTTTTGAAGGCCACTGGGAGGAGCCTGACCTAACCTTTTAGGGTGGTTAGGACATCTCCCCCACCTCCCCACTTTTTTCCCCAAGACAAGACAATCGAGGTCTGGCTTGAGAACGACCTTTCTTTCTTTATTTCTCAGCCTGCCCTTGGGGAGATGAGGGAGCCCTGTCTGCGTTTTTGGATGTGAGTAGAAGAGTTAGTTTGTTTTGTTTTATTATTCCTGGCCATACTCAGGGGTCCAGGAAGAATTTGTACCATTTAATGGGTTGGGAGTCTTGGCCAAGGAAGAATCACACCCTTGGAATAGAAATTTCCACCTCCCCAACCTTTCTCTCAGACAGCTTATCCTTTTCAACCAACTTTTTGGCCAGGGAGGAATGTCCCTTTTGTTCTTCCCCCTGAGAAGCCATTCCTTTGTCTGCCAACCTCCCTGGGGTCCTGCCTGTTTCCTCCCAATGGAGGGTTTTTTTGGGGGGTGGTCCCCGTCTGGGGGGCCCCTCCAGCCAGTACTCCAGGTCTCCCTGTCTCTCCCCCGCTGCCATTTTGATAGTATAATCTATTTTTAAATGGGGCTTTTCAATAGGGGAGAGGGAGTCATCTCTTCCTATATTTGGTGGGGTGGGTGGGAAGGAAGGGATTTGGGGGGGAATCTTCCTGCCGCCTCCCCCACTCCAAGTGTTTATTTTTGATACCAAACATGAATTTTCAGTTCCCTCCCTCCCAGCCCCCCAATTTCCTGCGGGCGGGTACAAAGGACCCTTTCAATGTCCCTGGAGTTGGGAGGGAGGAATGGGGGACATAAAGCCTGTCCTGTCTCTATTCTAGGCAAGAGAGAGTGGGTT"

    after_insert = "GTCACTGTTTGATGGCGAGCAAGACAGACTGGGTGACCAAAACCACATGTGAGAGACTACGTGGGCATGAAACTACAGTAGAGGTTGTCAGGGATTGTGGCAAAGATTGTGTAGGTGACAAACCTCACTGTAGGCGACTCAACGTTTGAGAAATAGATGATGTGCAAAACTGTTTACTTGAGGCTGTAGTGTCAGCTAAAGCTGTAAGACTGTATGGGTGCGAACTTGATTGTACGGGAGAGAATGGCTGGGAGACCAAACCAGTGACCGAGACCAAAGATACAAGAAGGTGCAATTGTGTGTACGAAAGCAGGCGTGAGAGACTGCAGCAAAGACTGAGAAAAACGAATGGCTGTGACTGGGAGAGTGTGACAGGCTGTGTGTGTGTGTGTGTACACCTGAAACTAGTTGCAAATCGATTTACCATTA"
    rest = "aactggatccgcaggcctctgctagcttgactgactgagatacagcgtaccttcagctcacgcggccgctttctccgcgatccagacatgataagatacattgatgagtttggacaaaccaaaactagaatgcagtgaaaaaaatgccttatttgtgaaatttgtgatgctattgccttatttgtaaccattataagctgcaataaacaagtt"
    

    
    dict = {}

    for keys, items in dict_insert_sequences.items():
        
        insert = items
        
        input_sequence = mGL + fifteen_mer_barcode + before_insert + insert + after_insert + rest
        
        acceptor_prob, donor_prob, nt_pos = get_probs(input_sequence)
        
        
        
        # I don't want to output the probability of each nucleotide
        # because I only want to output the probabilities that are greater than 0.01
        # by doing this, I can reduce the size of the output file
        acceptor_prob_list = acceptor_prob.tolist() # convert the numpy.arrays to list
        donor_prob_list = donor_prob.tolist()

        dict_temporary = {}
        
        for nt in nt_pos:
            
            dict_temporary[nt] = {'acceptor_prob': acceptor_prob_list[nt], 'donor_prob': donor_prob_list[nt]}

        list_nt = []
        list_prob = []
        
        for key, value in dict_temporary.items():
            
            acceptor_prob = value['acceptor_prob']
            donor_prob = value['donor_prob']
            
            # choose which probability is going to be used, acceptor or donor
            if acceptor_prob > donor_prob:
                
                prob = acceptor_prob
                
            else:
                
                prob = donor_prob
            
            if prob > 0.01:
                
                list_nt.append(key)
                list_prob.append(prob)
        
        
        
        dict[keys] = {'nt_pos': list_nt, 'prob': list_prob}
        
        print(keys)
    
    return dict

In [24]:
import csv
import gzip

def write_splicing_probabilities_to_csv(dict, output_csv):

    with gzip.open(output_csv, 'wt', newline='') as csvfile:
        fieldnames = ['No', 'nt_pos', 'prob']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        
        for key, items in dict.items():
            probabilities = items['prob']
            nt_positions = items['nt_pos']
            
            combined_data = zip([key]*len(nt_positions), nt_positions, probabilities)
            # [key]*len(nt_positions) creates a list of the same length as nt_positions, with each element being the key
            # bascially repeating the key for each nt_position and probabilities
            # zip() combines the lists into a list of tuples
            
            for row in combined_data:
                writer.writerow({'No': row[0], 'nt_pos': row[1], 'prob': row[2]})

In [19]:
insert_sequences_10_percent_first_replicate = extract_sequences_from_CSV_file('../1_generate candidate sequences/output/insert_sequences_of_10_percent_first_replicate.csv')

splicing_probability_10_percent_first_replicate = calculate_splicing_probabilities_of_insert_sequences(insert_sequences_10_percent_first_replicate)

write_splicing_probabilities_to_csv(splicing_probability_10_percent_first_replicate, 'full_library_splice_site_probabilities_10_percent_first_replicate.csv.gz')

1/1 [==============================] - 0s 172ms/step
10.0% - #1
1/1 [==============================] - 0s 140ms/step
10.0% - #2
1/1 [==============================] - 0s 140ms/step
10.0% - #3
1/1 [==============================] - 0s 148ms/step
10.0% - #4
1/1 [==============================] - 0s 145ms/step
10.0% - #5
1/1 [==============================] - 0s 138ms/step
10.0% - #6
1/1 [==============================] - 0s 140ms/step
10.0% - #7
1/1 [==============================] - 0s 144ms/step
10.0% - #8
1/1 [==============================] - 0s 143ms/step
10.0% - #9
1/1 [==============================] - 0s 140ms/step
10.0% - #10
1/1 [==============================] - 0s 138ms/step
10.0% - #11
1/1 [==============================] - 0s 160ms/step
10.0% - #12
1/1 [==============================] - 0s 136ms/step
10.0% - #13
1/1 [==============================] - 0s 141ms/step
10.0% - #14
1/1 [==============================] - 0s 143ms/step
10.0% - #15
1/1 [=============================

In [27]:
insert_sequences_10_percent_second_and_third_replicates = extract_sequences_from_CSV_file('../1_generate candidate sequences/output/insert_sequences_of_10_percent_second_and_third_replicates.csv')

splicing_probability_10_percent_second_and_third_replicates = calculate_splicing_probabilities_of_insert_sequences(insert_sequences_10_percent_second_and_third_replicates)

write_splicing_probabilities_to_csv(splicing_probability_10_percent_second_and_third_replicates, 'full_library_splice_site_probabilities_10_percent_second_and_third_replicates.csv.gz')

1/1 [==============================] - 0s 151ms/step
10.0% - #1
1/1 [==============================] - 0s 164ms/step
10.0% - #2
1/1 [==============================] - 0s 148ms/step
10.0% - #3
1/1 [==============================] - 0s 145ms/step
10.0% - #4
1/1 [==============================] - 0s 142ms/step
10.0% - #5
1/1 [==============================] - 0s 143ms/step
10.0% - #6
1/1 [==============================] - 0s 143ms/step
10.0% - #7
1/1 [==============================] - 0s 151ms/step
10.0% - #8
1/1 [==============================] - 0s 151ms/step
10.0% - #9
1/1 [==============================] - 0s 147ms/step
10.0% - #10
1/1 [==============================] - 0s 144ms/step
10.0% - #11
1/1 [==============================] - 0s 146ms/step
10.0% - #12
1/1 [==============================] - 0s 146ms/step
10.0% - #13
1/1 [==============================] - 0s 145ms/step
10.0% - #14
1/1 [==============================] - 0s 145ms/step
10.0% - #15
1/1 [=============================

In [29]:
insert_sequences_25_percent_first_replicate = extract_sequences_from_CSV_file('../1_generate candidate sequences/output/insert_sequences_of_25_percent_first_replicate.csv')

splicing_probability_25_percent_first_replicate = calculate_splicing_probabilities_of_insert_sequences(insert_sequences_25_percent_first_replicate)

write_splicing_probabilities_to_csv(splicing_probability_25_percent_first_replicate, 'full_library_splice_site_probabilities_25_percent_first_replicate.csv.gz')

1/1 [==============================] - 0s 147ms/step
25.0% - #1
1/1 [==============================] - 0s 142ms/step
25.0% - #2
1/1 [==============================] - 0s 143ms/step
25.0% - #3
1/1 [==============================] - 0s 142ms/step
25.0% - #4
1/1 [==============================] - 0s 141ms/step
25.0% - #5
1/1 [==============================] - 0s 143ms/step
25.0% - #6
1/1 [==============================] - 0s 142ms/step
25.0% - #7
1/1 [==============================] - 0s 141ms/step
25.0% - #8
1/1 [==============================] - 0s 141ms/step
25.0% - #9
1/1 [==============================] - 0s 140ms/step
25.0% - #10
1/1 [==============================] - 0s 144ms/step
25.0% - #11
1/1 [==============================] - 0s 140ms/step
25.0% - #12
1/1 [==============================] - 0s 139ms/step
25.0% - #13
1/1 [==============================] - 0s 142ms/step
25.0% - #14
1/1 [==============================] - 0s 141ms/step
25.0% - #15
1/1 [=============================

In [30]:
insert_sequences_25_percent_second_and_third_replicates = extract_sequences_from_CSV_file('../1_generate candidate sequences/output/insert_sequences_of_25_percent_second_and_third_replicates.csv')

splicing_probability_25_percent_second_and_third_replicates = calculate_splicing_probabilities_of_insert_sequences(insert_sequences_25_percent_second_and_third_replicates)

write_splicing_probabilities_to_csv(splicing_probability_25_percent_second_and_third_replicates, 'full_library_splice_site_probabilities_25_percent_second_and_third_replicates.csv.gz')

1/1 [==============================] - 0s 152ms/step
25.0% - #1
1/1 [==============================] - 0s 152ms/step
25.0% - #2
1/1 [==============================] - 0s 148ms/step
25.0% - #3
1/1 [==============================] - 0s 153ms/step
25.0% - #4
1/1 [==============================] - 0s 151ms/step
25.0% - #5
1/1 [==============================] - 0s 147ms/step
25.0% - #6
1/1 [==============================] - 0s 146ms/step
25.0% - #7
1/1 [==============================] - 0s 143ms/step
25.0% - #8
1/1 [==============================] - 0s 161ms/step
25.0% - #9
1/1 [==============================] - 0s 144ms/step
25.0% - #10
1/1 [==============================] - 0s 149ms/step
25.0% - #11
1/1 [==============================] - 0s 145ms/step
25.0% - #12
1/1 [==============================] - 0s 152ms/step
25.0% - #13
1/1 [==============================] - 0s 156ms/step
25.0% - #14
1/1 [==============================] - 0s 145ms/step
25.0% - #15
1/1 [=============================

In [31]:
insert_sequences_50_percent_first_replicate = extract_sequences_from_CSV_file('../1_generate candidate sequences/output/insert_sequences_of_50_percent_first_replicate.csv')

splicing_probability_50_percent_first_replicate = calculate_splicing_probabilities_of_insert_sequences(insert_sequences_50_percent_first_replicate)

write_splicing_probabilities_to_csv(splicing_probability_50_percent_first_replicate, 'full_library_splice_site_probabilities_50_percent_first_replicate.csv.gz')

1/1 [==============================] - 0s 149ms/step
50.0% - #1
1/1 [==============================] - 0s 148ms/step
50.0% - #2
1/1 [==============================] - 0s 149ms/step
50.0% - #3
1/1 [==============================] - 0s 145ms/step
50.0% - #4
1/1 [==============================] - 0s 144ms/step
50.0% - #5
1/1 [==============================] - 0s 148ms/step
50.0% - #6
1/1 [==============================] - 0s 146ms/step
50.0% - #7
1/1 [==============================] - 0s 155ms/step
50.0% - #8
1/1 [==============================] - 0s 147ms/step
50.0% - #9
1/1 [==============================] - 0s 151ms/step
50.0% - #10
1/1 [==============================] - 0s 152ms/step
50.0% - #11
1/1 [==============================] - 0s 148ms/step
50.0% - #12
1/1 [==============================] - 0s 145ms/step
50.0% - #13
1/1 [==============================] - 0s 146ms/step
50.0% - #14
1/1 [==============================] - 0s 146ms/step
50.0% - #15
1/1 [=============================

In [32]:
insert_sequences_50_percent_second_and_third_replicates = extract_sequences_from_CSV_file('../1_generate candidate sequences/output/insert_sequences_of_50_percent_second_and_third_replicates.csv')

splicing_probability_50_percent_second_and_third_replicates = calculate_splicing_probabilities_of_insert_sequences(insert_sequences_50_percent_second_and_third_replicates)

write_splicing_probabilities_to_csv(splicing_probability_50_percent_second_and_third_replicates, 'full_library_splice_site_probabilities_50_percent_second_and_third_replicates.csv.gz')

1/1 [==============================] - 0s 151ms/step
50.0% - #1
1/1 [==============================] - 0s 150ms/step
50.0% - #2
1/1 [==============================] - 0s 146ms/step
50.0% - #3
1/1 [==============================] - 0s 212ms/step
50.0% - #4
1/1 [==============================] - 0s 155ms/step
50.0% - #5
1/1 [==============================] - 0s 150ms/step
50.0% - #6
1/1 [==============================] - 0s 146ms/step
50.0% - #7
1/1 [==============================] - 0s 145ms/step
50.0% - #8
1/1 [==============================] - 0s 147ms/step
50.0% - #9
1/1 [==============================] - 0s 142ms/step
50.0% - #10
1/1 [==============================] - 0s 146ms/step
50.0% - #11
1/1 [==============================] - 0s 144ms/step
50.0% - #12
1/1 [==============================] - 0s 147ms/step
50.0% - #13
1/1 [==============================] - 0s 160ms/step
50.0% - #14
1/1 [==============================] - 0s 151ms/step
50.0% - #15
1/1 [=============================

In [33]:
insert_sequences_100_percent_three_replicates = extract_sequences_from_CSV_file('../1_generate candidate sequences/output/insert_sequences_of_100_percent_three_replicates.csv')

splicing_probability_100_percent_three_replicates = calculate_splicing_probabilities_of_insert_sequences(insert_sequences_100_percent_three_replicates)

write_splicing_probabilities_to_csv(splicing_probability_100_percent_three_replicates, 'full_library_splice_site_probabilities_100_percent_three_replicates.csv.gz')

1/1 [==============================] - 0s 156ms/step
100.0% - #1
1/1 [==============================] - 0s 141ms/step
100.0% - #2
1/1 [==============================] - 0s 144ms/step
100.0% - #3
1/1 [==============================] - 0s 145ms/step
100.0% - #4
1/1 [==============================] - 0s 149ms/step
100.0% - #5
1/1 [==============================] - 0s 143ms/step
100.0% - #6
1/1 [==============================] - 0s 145ms/step
100.0% - #7
1/1 [==============================] - 0s 147ms/step
100.0% - #8
1/1 [==============================] - 0s 141ms/step
100.0% - #9
1/1 [==============================] - 0s 148ms/step
100.0% - #10
1/1 [==============================] - 0s 152ms/step
100.0% - #11
1/1 [==============================] - 0s 144ms/step
100.0% - #12
1/1 [==============================] - 0s 142ms/step
100.0% - #13
1/1 [==============================] - 0s 147ms/step
100.0% - #14
1/1 [==============================] - 0s 142ms/step
100.0% - #15
1/1 [==============